In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import sys; sys.path.append("../")

from config import load_config
from utils import (load_data, preprocess, run_train, 
                   partition_train_val_test, print_eval_avg)



Using TensorFlow backend.


In [2]:
##Choose dataset
dataset = "BBBP"
#dataset = "BACE"
#dataset = "TOX21"

In [3]:
save_model = False

In [4]:
#Number of train/test/eval replicates to perform
N = 5

In [5]:
config = load_config(dataset)
data_fp = os.path.join(config['data_dir'], config['data_fn'])
raw_data = load_data(data_fp)
data = preprocess(raw_data)

100%|██████████| 2050/2050 [00:02<00:00, 817.13it/s]


In [ ]:
### Train / evaluate
eval_results = {}
for i in range(N):
    print("*"*50)
    print(i)
    inds = partition_train_val_test(raw_data["smiles"], dataset)
    model, eval_metrics = run_train(config, data, inds)
    eval_results[i] = eval_metrics
    
    if save_model:
        model_out_fn = "gcn_{}.h5".format(dataset.lower())
        model_out_fp = os.path.join(config["saved_models_dir"], model_out_fn) 

        if os.path.exists(model_out_fp):
            print("Model exists already. Skipping...")
        else:
            model.save(model_out_fp, overwrite=False)

In [ ]:
### Evaluate
print(dataset + "\n")

for metric in ["roc_auc", "avg_precision"]:
    print(metric)
    for split in ["train", "val  ", "test "]:
        res = print_eval_avg(eval_results, split.strip(), metric)
        print(split + " " + res)
    print()